# Autograd(자동미분)
*   torch.autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분 제공
*   이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻  
*   backprop를 위해 미분값을 자동으로 계산

requires_grad 속성을 True로 설정하면, 해당 텐서에서 이루어지는 모든 연산들을 추척하기 시작

기록을 추척하는 것을 중단하게 하려면, .detach()를 호출하여 연산 기록으로부터 분리


In [2]:
import torch
a = torch.randn(3,3)
a = a *3
print(a)
print(a.requires_grad) # False

tensor([[-0.8107,  0.9547,  2.0495],
        [ 2.4820,  1.4881,  5.4323],
        [-4.6738, -0.1096,  1.2293]])
False


requires_grad_(...)는 기존 텐서의 requires_grad 값을 바꿔치기(in-place)하여 변경

grad_fn: 미분 값을 계산한 함수에 대한 정보 저장( 어떤 함수에 대해서 backprop 했는지):

In [3]:
a.requires_grad_(True)
print(a.requires_grad)
b = (a*a)
print(b)
b = (a*a).sum()
print(b) # 모든 값을 더한것
print(b.grad_fn) # 모든 값을 더한것
# tensor(96.2691, grad_fn=<SumBackward0>) -> 이렇게 기록으로 남기는 것

True
tensor([[6.5728e-01, 9.1136e-01, 4.2004e+00],
        [6.1605e+00, 2.2144e+00, 2.9510e+01],
        [2.1845e+01, 1.2004e-02, 1.5111e+00]], grad_fn=<MulBackward0>)
tensor(67.0215, grad_fn=<SumBackward0>)


# 기울기(Gradient)

In [4]:
x = torch.ones(3,3,requires_grad=True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [5]:
y = x +5
print(y)

tensor([[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]], grad_fn=<AddBackward0>)


In [6]:
z=y*y
out = z.mean()
print(z,out)
# MulBackward0 # 곱에 대한 그래디언트 저장
# Mean

tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]], grad_fn=<MulBackward0>) tensor(36., grad_fn=<MeanBackward0>)


계산이 완료된 후, backward()를 호출하면 자동으로 역전파 계산이 가능하고, .grad 속성에 누적됌

In [7]:
print(out) # 호출
out.backward()
# backward() 메서드를 호출하면,
# PyTorch는 미적분학의 연쇄 법칙을 사용하여 기울기를 계산하기 위해
# 이 그래프를 역순으로 (출력에서 입력까지) 순회합니다.

tensor(36., grad_fn=<MeanBackward0>)


grad : data가 거쳐온 layer에 대한 미분값 저장

In [8]:
print(x)
print(x.grad) # 미분 값이 몇인지 알려줘

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])


In [9]:
x = torch.randn(3,requires_grad=True)
y = x *2
while y.data.norm()<1000 : # 노멀라이제이션 1000이하라면
  y = y*2
print(y)

tensor([-1132.2120,   797.5872,  -964.3265], grad_fn=<MulBackward0>)


In [10]:
v = torch.tensor([0.1,1.0,0.0001],dtype=torch.float) # 외부 그래디언트
y.backward(v) # v를 기준으로 백워드가 된다.
print(x.grad) # 그래디언트

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])



# with torch.no_grad()
with torch.no_grad()를 사용하여 기울기의 업데이트를 하지 않음

기록을 추척하는 것을 방지하기 위해 코드 블랙을 with torch.no_Grad()로 감싸면 기울기 계산은 필요 없지만,

 requires_grad =True로 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용

In [11]:
# 모델을 평가할 때만 사용
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad() : # 기울기 업데이트를 하지 말아줘
  print((x**2).requires_grad)

True
True
False


detach() : 내용물(content)은 같지만 require_grad가 다른 새로운 Tensor를 가져올때

In [13]:
'''
detach() 메서드는 PyTorch의 텐서에서 매우 중요한 기능을 수행합니다.
이 메서드를 사용하여 텐서를 원래의 계산 그래프에서 "분리"할 수 있습니다.

detach()에 대한 주요 사항

분리된 텐서 생성: detach() 메서드는 원본 텐서와 동일한 데이터를 가지지만,
계산 그래프에서 분리된 새 텐서를 반환합니다.

기울기 추적 비활성화: detach()로 생성된 텐서는 기울기 계산에서 제외됩니다.
즉, 분리된 텐서의 requires_grad 속성은 False로 설정됩니다.

메모리 공유: 원본 텐서와 detach()로 생성된 텐서는 동일한 메모리를 공유합니다.
따라서, 하나의 텐서의 값을 변경하면 다른 텐서의 값도 변경됩니다.

모델 평가 및 변환: detach()는 주로 모델을 평가할 때나 텐서 값을 numpy 배열로 변환할 때 사용됩니다.
이러한 경우에는 기울기를 계산할 필요가 없기 때문에 텐서를 계산 그래프에서 분리하여 기울기 계산을 방지합니다.
즉 미분 안하게 하기 위해서 그냥 값만 쓰윽 빼온것
'''
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all()) # y와 x가 같은지 모두 계싼


True
False
tensor(True)
